<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

# Session 2: Modelado de Optimización - Habitat Adaptation v0

**Objetivo:** Diseñar y resolver un modelo MILP.

**Fecha:** 29 de octubre de 2025

## 1. Importaciones

In [1]:
import sys, os, json
from datetime import datetime
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from pyomo.environ import *
from pyomo.opt import SolverFactory, SolverStatus, TerminationCondition

print('✓ Librerías importadas')

✓ Librerías importadas


## 2. Carga de Dataset

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
dataset_path = '../../data/dataset_processed.geojson'
gdf = gpd.read_file(dataset_path)
print(f'✓ Dataset cargado: {len(gdf)} celdas')
print(f'✓ Columnas: {len(gdf.columns)}')
print(gdf[['grid_id', 'cost_adaptation_atelerix', 'has_atelerix_algirus']].head())

DataSourceError: ../../data/dataset_processed.geojson: No such file or directory

## 3. Parámetros del Modelo

In [ ]:
SPECIES = {
    'atelerix': 'has_atelerix_algirus',
    'martes': 'has_martes_martes',
    'eliomys': 'has_eliomys_quercinus',
    'oryctolagus': 'has_oryctolagus_cuniculus'
}

COST_COLS = {
    'atelerix': 'cost_adaptation_atelerix',
    'martes': 'cost_adaptation_martes',
    'eliomys': 'cost_adaptation_eliomys',
    'oryctolagus': 'cost_adaptation_oryctolagus'
}

cells = gdf['grid_id'].tolist()
species_list = list(SPECIES.keys())

h, c = {}, {}
for idx, row in gdf.iterrows():
    cell_id = row['grid_id']
    for sp in species_list:
        h[(cell_id, sp)] = int(row[SPECIES[sp]])
        c[(cell_id, sp)] = float(row[COST_COLS[sp]])

print(f'✓ Celdas: {len(cells)}, Especies: {species_list}')
print(f'✓ Hábitats actuales: {sum(h.values())}')
print(f'✓ Rango de costes: [{min(c.values()):.2f}, {max(c.values()):.2f}]')

## 4. Definición del Modelo MILP

In [ ]:
def create_habitat_model(cells, species_list, c, h, budget, weights=None):
    if weights is None:
        weights = {sp: 1.0 for sp in species_list}
    
    model = ConcreteModel()
    model.CELLS = Set(initialize=cells)
    model.SPECIES = Set(initialize=species_list)
    model.budget = Param(initialize=budget)
    model.cost = Param(model.CELLS, model.SPECIES, initialize=c, default=0)
    model.habitat = Param(model.CELLS, model.SPECIES, initialize=h, default=0)
    model.weight = Param(model.SPECIES, initialize=weights, default=1.0)
    model.x = Var(model.CELLS, model.SPECIES, within=Binary, initialize=0)
    
    def objective_expr(model):
        return sum(model.weight[s] * (model.habitat[i, s] + model.x[i, s])
                  for i in model.CELLS for s in model.SPECIES)
    model.obj = Objective(expr=objective_expr, sense=maximize)
    
    def budget_constraint(model):
        return sum(model.cost[i, s] * model.x[i, s]
                  for i in model.CELLS for s in model.SPECIES) <= model.budget
    model.budget_constr = Constraint(rule=budget_constraint)
    
    def no_duplication(model, i, s):
        return model.x[i, s] <= (1 - model.habitat[i, s])
    model.no_dup_constr = Constraint(model.CELLS, model.SPECIES, rule=no_duplication)
    
    return model

print('✓ Función de modelo definida')

## 5. Resolución del Modelo

In [ ]:
BUDGET = 500.0
weights = {'atelerix': 1.0, 'martes': 1.2, 'eliomys': 1.5, 'oryctolagus': 0.8}

print(f'Presupuesto: {BUDGET}\nPesos: {weights}\n')

model = create_habitat_model(cells, species_list, c, h, BUDGET, weights)
print('✓ Modelo instanciado')

print('🔄 Resolviendo...')
solver = SolverFactory('glpk')
results = solver.solve(model, tee=False)

if results.solver.termination_condition == TerminationCondition.optimal:
    print('✓ SOLUCIÓN ÓPTIMA ENCONTRADA')
    is_optimal = True
else:
    print('⚠ Solución factible')
    is_optimal = False

print(f'Valor objetivo: {value(model.obj):.2f}')

## 6. Extracción de Resultados

In [ ]:
adaptations = []
for i in model.CELLS:
    for s in model.SPECIES:
        if value(model.x[i, s]) > 0.5:
            adaptations.append({
                'grid_id': i, 'species': s, 'cost': c[(i, s)],
                'current_habitat': h[(i, s)]
            })

adaptations_df = pd.DataFrame(adaptations)
total_cost = adaptations_df['cost'].sum()

print(f'✓ Celdas adaptadas: {len(adaptations_df)}')
print(f'Coste total: {total_cost:.2f} / {BUDGET:.2f}')
print(f'Eficiencia: {(total_cost/BUDGET)*100:.1f}%')

## 7. Análisis por Especie

In [ ]:
print('DESGLOSE POR ESPECIE:')
for sp in species_list:
    sp_data = adaptations_df[adaptations_df['species'] == sp]
    current = sum(h.get((i, sp), 0) for i in cells)
    adapted = len(sp_data)
    total = current + adapted
    cost_sp = sp_data['cost'].sum()
    print(f'\n{sp}: {current} + {adapted} = {total} ({(total/len(cells)*100):.1f}%)')
    print(f'  Coste: {cost_sp:.2f}')

## 8. Guardado de Resultados

In [ ]:
output_dir = '../../data'
os.makedirs(output_dir, exist_ok=True)

adaptations_df.to_csv(f'{output_dir}/adaptations_detailed_v0.csv', index=False)
print(f'✓ Guardado: {output_dir}/adaptations_detailed_v0.csv')

solution_metadata = {
    'session': 'Session 2',
    'model_version': 'v0_habitat_adaptation',
    'date': datetime.now().isoformat(),
    'budget': BUDGET,
    'objective_value': float(value(model.obj)),
    'total_cost': float(total_cost),
    'n_adaptations': len(adaptations_df),
    'solution_type': 'optimal' if is_optimal else 'feasible'
}

with open(f'{output_dir}/solution_metadata_v0.json', 'w') as f:
    json.dump(solution_metadata, f, indent=2)
print(f'✓ Guardado: {output_dir}/solution_metadata_v0.json')

print('\n✅ Session 2 completada!')